In [20]:
import requests
from bs4 import BeautifulSoup
import csv

def scrape():
    url = 'https://www.goal.com/tr/sonu%C3%A7lar/2024-11-11'
    
    # Send the HTTP request to the page
    response = requests.get(url)
    
    # Check if the request was successful (status code 200)
    if response.status_code != 200:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")
        return
    
    # Parse the page content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Try to find the match date
    match_date_tag = soup.find('h1', class_='header_header__x_52R header')
    if not match_date_tag:
        print("Match date not found.")
        return
    
    match_date = match_date_tag.text.strip()

    # Open the CSV file to write data
    with open('match_results.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)

        # Write the header to the CSV file
        writer.writerow(['Match Date', 'Home Team', 'Away Team', 'Home Score', 'Away Score'])

        # Find all the matches
        matches = soup.find_all('div', class_='live-scores_live-scores-wrapper__lGxsb')
        
        # Check if we found any matches
        if not matches:
           print("No match data found.:()")
           return
        
        # Loop through each match and extract the details
      
        for match in matches:
            try:
                # Home team
                home_team_tag = match.find('div', class_='team_team__CSYuy team_team-a__KZ1AE').find('h4')
                home_team = home_team_tag.text.strip() if home_team_tag else "N/A"
                
                # Away team
                away_team_tag = match.find('div', class_='team_team__CSYuy team_team-b__6xMTs').find('h4')
                away_team = away_team_tag.text.strip() if away_team_tag else "N/A"
                
                # Score
                score_tag = match.find('div', class_='result_result__J9gCz').find('div', class_='result_score__yfsC8')
                if score_tag:
                    home_score_tag = score_tag.find('p', class_='result_team-a__jx1EM')
                    away_score_tag = score_tag.find('p', class_='result_team-b__kNMbF')
                    
                    home_score = home_score_tag.text.strip() if home_score_tag else "N/A"
                    away_score = away_score_tag.text.strip() if away_score_tag else "N/A"
                else:
                    home_score, away_score = "N/A", "N/A"
                
                # Write match details to CSV
                writer.writerow([match_date, home_team, away_team, home_score, away_score])
            
            except Exception as e:
                print(f"Error parsing match data: {e}")
    
    print("Data scraping completed and saved to match_results.csv")

if __name__ == '__main__':
    scrape()


No match data found.opjıhkjg


In [31]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv

# Set up WebDriver for Firefox with Selenium Manager
options = webdriver.FirefoxOptions()
options.headless = True  # Run in headless mode
driver = webdriver.Firefox(options=options)

# Define the range of years and weeks to iterate over
years1 = range(2017, 2025)  # This will go from 2017 to 2024
matchDayWeeks = range(1, 35)

# Define an empty list to hold all match data
all_matches = []

# Loop through each year and week
for year in years1:
    next_year = year + 1  # Calculate the next year for the URL
    for week in matchDayWeeks:
        # Construct the URL with the correct year range and week
        url = f'https://www.bundesliga.com/en/bundesliga/matchday/{year}-{next_year}/{week}'
        driver.get(url)
        timeout=5#some time

        # Wait for the match rows to load
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '#fixtures > fixturescomponent'))
            )
        except Exception as e:
            print(f"Timeout or error loading page for year {year}, week {week}: {e}")
            continue

        match_rows = driver.find_elements(By.CSS_SELECTOR, '#fixtures > fixturescomponent > div')
        
        for row in match_rows:
            try:
                date = row.find_element(By.CSS_SELECTOR, '#fixtures > fixturescomponent > div > match-date-header:nth-child(1) > div').text
                home_team = row.find_element(By.CSS_SELECTOR, '#fixtures > fixturescomponent > div > div:nth-child(2) > a > match-fixture > match-team:nth-child(1) > div > div.name.d-none.d-lg-none.d-xl-none.d-md-block').text
                away_team = row.find_element(By.CSS_SELECTOR, 't#fixtures > fixturescomponent > div > div:nth-child(2) > a > match-fixture > match-team:nth-child(3) > div > div.name.d-none.d-lg-none.d-xl-none.d-md-block').text
                result = row.find_element(By.CSS_SELECTOR, '#fixtures > fixturescomponent > div > div:nth-child(2) > a > match-fixture > score-bug > div').text

                all_matches.append({
                    'year': year,
                    'week': week,
                    'date': date,
                    'home_team': home_team,
                    'away_team': away_team,
                    'result': result
                })
            except Exception as e:
                print(f"Error extracting data for year {year}, week {week}: {e}")
                continue

driver.quit()

# Save all data to CSV
csv_file = 'tff_match_data_2000_2024.csv'
csv_columns = ['year', 'week', 'date', 'home_team', 'away_team', 'result']

with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=csv_columns)
    writer.writeheader()
    writer.writerows(all_matches)

print(f"Data saved to {csv_file}")

Error extracting data for year 2017, week 1: Message: Unable to locate element: t#fixtures > fixturescomponent > div > div:nth-child(2) > a > match-fixture > match-team:nth-child(3) > div > div.name.d-none.d-lg-none.d-xl-none.d-md-block; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

Error extracting data for year 2017, week 2: Message: Unable to locate element: t#fixtures > fixturescomponent > div > div:nth-child(2) > a > match-fixture > match-team:nth-child(3) > div > div.name.d-none.d-lg-none.d-xl-none.d-md-block; For documentation on this error, please visit: https://www.selenium.dev/docume

## Bundesliga

In [43]:
#Bundesligaa 
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv

# Set up WebDriver for Firefox with Selenium Manager
options = webdriver.FirefoxOptions()
options.headless = True  # Run in headless mode
driver = webdriver.Firefox(options=options)

# Define the range of years and weeks to iterate over
years1 = range(2017, 2025)  # This will go from 2017 to 2024
matchDayWeeks = range(1, 35)

# Define an empty list to hold all match data
all_matches = []

# Loop through each year and week
for year in years1:
    next_year = year + 1  # Calculate the next year for the URL
    for week in matchDayWeeks:
        # Construct the URL with the correct year range and week
        url = f'https://www.bundesliga.com/en/bundesliga/matchday/{year}-{next_year}/{week}'
        driver.get(url)

        # Wait for the main component to load
        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'fixturescomponent'))
            )
        except Exception as e:
            print(f"Timeout or error loading page for year {year}, week {week}: {e}")
            continue

        # Select match rows within the fixture component
        match_rows = driver.find_elements(By.CSS_SELECTOR, 'fixturescomponent > div.matchRow')
        
        # Extract match data from each row
        for row in match_rows:
            try:
                # Date of the match
                date = row.find_element(By.CSS_SELECTOR, 'match-date-header span.day').text
                
                # Team names
                home_team = row.find_element(By.CSS_SELECTOR, 'match-team[side="home"] .name.d-md-block').text
                away_team = row.find_element(By.CSS_SELECTOR, 'match-team[side="away"] .name.d-md-block').text
                
                # Score
                home_score = row.find_element(By.CSS_SELECTOR, 'score-bug .cell.home .score').text
                away_score = row.find_element(By.CSS_SELECTOR, 'score-bug .cell.away .score').text
                result = f"{home_score}-{away_score}"

                # Append data to the list
                all_matches.append({
                    'year': year,
                    'week': week,
                    'date': date,
                    'home_team': home_team,
                    'away_team': away_team,
                    'result': result
                })
            except Exception as e:
                print(f"Error extracting data for year {year}, week {week}: {e}")
                continue

driver.quit()

# Save all data to CSV
csv_file = 'bundesliga_match_data.csv'
csv_columns = ['year', 'week', 'date', 'home_team', 'away_team', 'result']

with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=csv_columns)
    writer.writeheader()
    writer.writerows(all_matches)

print(f"Data saved to {csv_file}")


Timeout or error loading page for year 2020, week 12: Message: Browsing context has been discarded
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchWindowError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:679:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.sys.mjs:515:13
assert.open@chrome://remote/content/shared/webdriver/Assert.sys.mjs:147:4
GeckoDriver.prototype.findElement@chrome://remote/content/marionette/driver.sys.mjs:1539:15
despatch@chrome://remote/content/marionette/server.sys.mjs:318:40
execute@chrome://remote/content/marionette/server.sys.mjs:289:16
onPacket/<@chrome://remote/content/marionette/server.sys.mjs:262:20
onPacket@chrome://remote/content/marionette/server.sys.mjs:263:9
_onJSONObjectReady/<@chrome://remote/content/marionette/transport.sys.mjs:494:20



NoSuchWindowException: Message: Browsing context has been discarded
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchWindowError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:679:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.sys.mjs:515:13
assert.open@chrome://remote/content/shared/webdriver/Assert.sys.mjs:147:4
GeckoDriver.prototype.navigateTo@chrome://remote/content/marionette/driver.sys.mjs:859:39
despatch@chrome://remote/content/marionette/server.sys.mjs:318:40
execute@chrome://remote/content/marionette/server.sys.mjs:289:16
onPacket/<@chrome://remote/content/marionette/server.sys.mjs:262:20
onPacket@chrome://remote/content/marionette/server.sys.mjs:263:9
_onJSONObjectReady/<@chrome://remote/content/marionette/transport.sys.mjs:494:20


In [42]:
#MAçkolikkkkk
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time

# Set up WebDriver for Firefox with Selenium Manager
options = webdriver.FirefoxOptions()
options.headless = True  # Run in headless mode
driver = webdriver.Firefox(options=options)

# Define the seasons to scrape
years = range(1999, 2025)  # Adjust the range for required seasons

# Define an empty list to hold all match data
all_matches = []

# Loop through each season
for year in years:
    next_year = year + 1
    url = f'https://www.mackolik.com/puan-durumu/t%C3%BCrkiye-trendyol-s%C3%BCper-lig/{year}-{next_year}/fikstur/482ofyysbdbeoxauk19yg7tdt'
    driver.get(url)
    time.sleep(2)  # Pause briefly to allow initial page load

    # Loop through each of the 34 weeks
    week_loaded = 0  # Track number of weeks loaded
    while week_loaded < 34:
        try:
            # Wait for the "Load More" or week navigation button and click it
            load_more_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, 'body > div.page-container.page-container--legacy-link-banner-visible > div.main-container > main > div.page-competition-index > div.page-competition-fixtures-index__container.page-competition-fixtures-index__container--date-selection > div > div > div.widget-gameweek__selected-label'))
            )
            load_more_button.click()
            time.sleep(2)  # Wait for the page to load the new week’s data

            # Increment the counter if the button was clicked successfully
            week_loaded += 1
            print(f"Loaded week {week_loaded} for season {year}-{next_year}")
        except Exception as e:
            print(f"Could not load week {week_loaded + 1} for season {year}-{next_year}: {e}")
            break

        # Extract match data after each click (each loaded week)
        try:
            fixture_rows = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'body > div.page-container.page-container--legacy-link-banner-visible > div.main-container > main > div.page-competition-index > div.page-competition-index__container > div.page-competition-fixtures-index > div > div'))
            )
            for row in fixture_rows:
                date = row.find_element(By.CSS_SELECTOR, 'body > div.page-container.page-container--legacy-link-banner-visible > div.main-container > main > div.page-competition-index > div.page-competition-index__container > div.page-competition-fixtures-index > div > div > ol > li:nth-child(1) > div > span.p0c-competition-match-list__title-date').text
                home_team = row.find_element(By.CSS_SELECTOR, 'body > div.page-container.page-container--legacy-link-banner-visible > div.main-container > main > div.page-competition-index > div.page-competition-index__container > div.page-competition-fixtures-index > div > div > ol > li:nth-child(4) > ol > li > div > div.p0c-competition-match-list__teams > div.p0c-competition-match-list__team.p0c-competition-match-list__team--home > a.p0c-competition-match-list__team-name.p0c-competition-match-list__team-name--home > span').text
                away_team = row.find_element(By.CSS_SELECTOR, 'body > div.page-container.page-container--legacy-link-banner-visible > div.main-container > main > div.page-competition-index > div.page-competition-index__container > div.page-competition-fixtures-index > div > div > ol > li:nth-child(4) > ol > li > div > div.p0c-competition-match-list__teams > div.p0c-competition-match-list__team.p0c-competition-match-list__team--away > a.p0c-competition-match-list__team-name.p0c-competition-match-list__team-name--away > span').text
                home_score = row.find_element(By.CSS_SELECTOR, 'body > div.page-container.page-container--legacy-link-banner-visible > div.main-container > main > div.page-competition-index > div.page-competition-index__container > div.page-competition-fixtures-index > div > div > ol > li:nth-child(1) > ol > li > div > div.p0c-competition-match-list__teams > div.p0c-competition-match-list__team.p0c-competition-match-list__team--home > a.p0c-competition-match-list__match-link > span').text
                away_score = row.find_element(By.CSS_SELECTOR, 'body > div.page-container.page-container--legacy-link-banner-visible > div.main-container > main > div.page-competition-index > div.page-competition-index__container > div.page-competition-fixtures-index > div > div > ol > li:nth-child(1) > ol > li > div > div.p0c-competition-match-list__teams > div.p0c-competition-match-list__team.p0c-competition-match-list__team--away > a.p0c-competition-match-list__match-link > span').text
                result = f"{home_score}-{away_score}"

                # Append data to the list
                all_matches.append({
                    'year': f"{year}-{next_year}",
                    'week': week_loaded,
                    'date': date,
                    'home_team': home_team,
                    'away_team': away_team,
                    'result': result
                })
        except Exception as e:
            print(f"Error extracting data for {year}-{next_year}, week {week_loaded}: {e}")
            continue

# Close the WebDriver
driver.quit()

# Save all data to CSV
csv_file = 'super_lig_match_data.csv'
csv_columns = ['year', 'week', 'date', 'home_team', 'away_team', 'result']

with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=csv_columns)
    writer.writeheader()
    writer.writerows(all_matches)

print(f"Data saved to {csv_file}")


Loaded week 1 for season 1999-2000
Loaded week 2 for season 1999-2000
Loaded week 3 for season 1999-2000
Loaded week 4 for season 1999-2000
Loaded week 5 for season 1999-2000
Loaded week 6 for season 1999-2000
Loaded week 7 for season 1999-2000
Loaded week 8 for season 1999-2000
Loaded week 9 for season 1999-2000
Loaded week 10 for season 1999-2000
Loaded week 11 for season 1999-2000
Loaded week 12 for season 1999-2000
Error extracting data for 1999-2000, week 12: Message: Browsing context has been discarded
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchWindowError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:679:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.sys.mjs:515:13
assert.open@chrome://remote/content/shared/webdriver/Assert.sys.mjs:147:4
GeckoDriver.prototype.findElements@chrome://remote/content/marionette/driver.sys.mjs:1638:15
despat

NoSuchWindowException: Message: Browsing context has been discarded
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchWindowError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:679:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.sys.mjs:515:13
assert.open@chrome://remote/content/shared/webdriver/Assert.sys.mjs:147:4
GeckoDriver.prototype.navigateTo@chrome://remote/content/marionette/driver.sys.mjs:859:39
despatch@chrome://remote/content/marionette/server.sys.mjs:318:40
execute@chrome://remote/content/marionette/server.sys.mjs:289:16
onPacket/<@chrome://remote/content/marionette/server.sys.mjs:262:20
onPacket@chrome://remote/content/marionette/server.sys.mjs:263:9
_onJSONObjectReady/<@chrome://remote/content/marionette/transport.sys.mjs:494:20


In [44]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time

# Set up WebDriver for Firefox with Selenium Manager
options = webdriver.FirefoxOptions()
options.headless = True  # Run in headless mode for efficiency
driver = webdriver.Firefox(options=options)

# Define the range of years and matchdays to scrape
years = range(2017, 2025)  # From 2017-2018 to 2024-2025 seasons
match_days = range(1, 35)  # Typical matchdays in a season

# List to store all match data
all_matches = []

for year in years:
    next_year = year + 1  # Bundesliga season spans two calendar years
    for week in match_days:
        url = f'https://www.bundesliga.com/en/bundesliga/matchday/{year}-{next_year}/{week}'
        driver.get(url)
        print(f"Scraping: {url}")
        
        try:
            # Wait for fixtures to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'fixturescomponent'))
            )

            # Get all match rows
            match_rows = driver.find_elements(By.CSS_SELECTOR, 'fixturescomponent .matchRow')
            
            for row in match_rows:
                try:
                    date = row.find_element(By.CSS_SELECTOR, 'match-date-header span.day').text.strip()
                    home_team = row.find_element(By.CSS_SELECTOR, 'match-team[side="home"] .name.d-md-block').text.strip()
                    away_team = row.find_element(By.CSS_SELECTOR, 'match-team[side="away"] .name.d-md-block').text.strip()
                    home_score = row.find_element(By.CSS_SELECTOR, 'score-bug .cell.home .score').text.strip()
                    away_score = row.find_element(By.CSS_SELECTOR, 'score-bug .cell.away .score').text.strip()
                    result = f"{home_score}-{away_score}"
                    
                    # Store match data
                    all_matches.append({
                        'season': f"{year}-{next_year}",
                        'match_day': week,
                        'date': date,
                        'home_team': home_team,
                        'away_team': away_team,
                        'result': result
                    })
                except Exception as e:
                    print(f"Error extracting data for match: {e}")
        except Exception as e:
            print(f"Failed to load matchday {week} for {year}-{next_year}: {e}")
        time.sleep(1)  # Avoid hitting the server too quickly

# Close the WebDriver
driver.quit()

# Save data to CSV
csv_file = 'bundesliga_full_data.csv'
csv_columns = ['season', 'match_day', 'date', 'home_team', 'away_team', 'result']

with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=csv_columns)
    writer.writeheader()
    writer.writerows(all_matches)

print(f"Data saved to {csv_file}")


Scraping: https://www.bundesliga.com/en/bundesliga/matchday/2017-2018/1
Error extracting data for match: Message: Unable to locate element: match-date-header span.day; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

Error extracting data for match: Message: Unable to locate element: match-date-header span.day; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys

NoSuchWindowException: Message: Browsing context has been discarded
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchWindowError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:679:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.sys.mjs:515:13
assert.open@chrome://remote/content/shared/webdriver/Assert.sys.mjs:147:4
GeckoDriver.prototype.navigateTo@chrome://remote/content/marionette/driver.sys.mjs:859:39
despatch@chrome://remote/content/marionette/server.sys.mjs:318:40
execute@chrome://remote/content/marionette/server.sys.mjs:289:16
onPacket/<@chrome://remote/content/marionette/server.sys.mjs:262:20
onPacket@chrome://remote/content/marionette/server.sys.mjs:263:9
_onJSONObjectReady/<@chrome://remote/content/marionette/transport.sys.mjs:494:20


In [47]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time

# Set up WebDriver for Firefox with Selenium Manager
options = webdriver.FirefoxOptions()
options.headless = True  # Run in headless mode for efficiency
driver = webdriver.Firefox(options=options)

# Define the range of years and matchdays to scrape
years = range(2017, 2025)  # From 2017-2018 to 2024-2025 seasons
match_days = range(1, 35)  # Typical matchdays in a season

# List to store all match data
all_matches = []

for year in years:
    next_year = year + 1  # Bundesliga season spans two calendar years
    for week in match_days:
        url = f'https://www.bundesliga.com/en/bundesliga/matchday/{year}-{next_year}/{week}'
        driver.get(url)
        print(f"Scraping: {url}")
        
        try:
            # Wait for the fixtures component to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'fixturescomponent'))
            )

            # Re-inspect and verify that the selectors below match the page structure
            match_rows = driver.find_elements(By.CSS_SELECTOR, 'fixturescomponent .matchRow')

            for row in match_rows:
                try:
                    # Adjust selectors as per inspected page
                    date = row.find_element(By.CSS_SELECTOR, '##fixtures > fixturescomponent > div > match-date-header:nth-child(1) > div > span.day.ng-star-inserted').text.strip()
                    home_team = row.find_element(By.CSS_SELECTOR, 'match-team[side="home"] .name').text.strip()
                    away_team = row.find_element(By.CSS_SELECTOR, 'match-team[side="away"] .name').text.strip()
                    home_score = row.find_element(By.CSS_SELECTOR, 'score-bug .cell.home .score').text.strip()
                    away_score = row.find_element(By.CSS_SELECTOR, 'score-bug .cell.away .score').text.strip()
                    result = f"{home_score}-{away_score}"
                    
                    all_matches.append({
                        'season': f"{year}-{next_year}",
                        'match_day': week,
                        'date': date,
                        'home_team': home_team,
                        'away_team': away_team,
                        'result': result
                    })
                except Exception as e:
                    print(f"Error extracting data for match: {e}")
        except Exception as e:
            print(f"Failed to load matchday {week} for {year}-{next_year}: {e}")
        time.sleep(1)  # Delay to avoid server issues

# Close the WebDriver
driver.quit()

# Save data to CSV
csv_file = 'bundesliga_full_data.csv'
csv_columns = ['season', 'match_day', 'date', 'home_team', 'away_team', 'result']

with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=csv_columns)
    writer.writeheader()
    writer.writerows(all_matches)

print(f"Data saved to {csv_file}")


Scraping: https://www.bundesliga.com/en/bundesliga/matchday/2017-2018/1
Error extracting data for match: Message: Given css selector expression "##fixtures > fixturescomponent > div > match-date-header:nth-child(1) > div > span.day.ng-star-inserted" is invalid: InvalidSelectorError: Element.querySelector: '##fixtures > fixturescomponent > div > match-date-header:nth-child(1) > div > span.day.ng-star-inserted' is not a valid selector: "##fixtures > fixturescomponent > div > match-date-header:nth-child(1) > div > span.day.ng-star-inserted"; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalid-selector-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
InvalidSelectorError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:432:5
find_@chrome://remote/content/shared/DOM.sys.mjs:170:11

Error extract

NoSuchWindowException: Message: Browsing context has been discarded
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchWindowError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:679:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.sys.mjs:515:13
assert.open@chrome://remote/content/shared/webdriver/Assert.sys.mjs:147:4
GeckoDriver.prototype.navigateTo@chrome://remote/content/marionette/driver.sys.mjs:859:39
despatch@chrome://remote/content/marionette/server.sys.mjs:318:40
execute@chrome://remote/content/marionette/server.sys.mjs:289:16
onPacket/<@chrome://remote/content/marionette/server.sys.mjs:262:20
onPacket@chrome://remote/content/marionette/server.sys.mjs:263:9
_onJSONObjectReady/<@chrome://remote/content/marionette/transport.sys.mjs:494:20


In [48]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time

# Set up WebDriver for Firefox with Selenium Manager
options = webdriver.FirefoxOptions()
options.headless = True
driver = webdriver.Firefox(options=options)

# Define the range of years and matchdays
years = range(2017, 2025)
match_days = range(1, 35)

# List to store all match data
all_matches = []

def get_element_text(driver, selector):
    """Helper function to get element text safely."""
    try:
        return driver.find_element(By.CSS_SELECTOR, selector).text.strip()
    except Exception as e:
        print(f"Selector {selector} failed: {e}")
        return ""

for year in years:
    next_year = year + 1
    for week in match_days:
        url = f'https://www.bundesliga.com/en/bundesliga/matchday/{year}-{next_year}/{week}'
        driver.get(url)
        print(f"Scraping: {url}")

        try:
            # Wait for fixtures component to load
            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'fixturescomponent'))
            )

            # Identify match rows
            match_rows = driver.find_elements(By.CSS_SELECTOR, 'fixturescomponent .matchRow')
            if not match_rows:
                print(f"No matches found for {year}-{next_year}, week {week}")
                continue

            for row in match_rows:
                try:
                    date = get_element_text(row, 'match-date-header span.day')
                    home_team = get_element_text(row, 'match-team[side="home"] .name')
                    away_team = get_element_text(row, 'match-team[side="away"] .name')
                    home_score = get_element_text(row, 'score-bug .cell.home .score')
                    away_score = get_element_text(row, 'score-bug .cell.away .score')
                    result = f"{home_score}-{away_score}"

                    all_matches.append({
                        'season': f"{year}-{next_year}",
                        'match_day': week,
                        'date': date,
                        'home_team': home_team,
                        'away_team': away_team,
                        'result': result
                    })
                except Exception as e:
                    print(f"Error extracting match data: {e}")

        except Exception as e:
            print(f"Error loading {url}: {e}")
        time.sleep(1)

driver.quit()

# Save to CSV
csv_file = 'bundesliga_full_data.csv'
csv_columns = ['season', 'match_day', 'date', 'home_team', 'away_team', 'result']

with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=csv_columns)
    writer.writeheader()
    writer.writerows(all_matches)

print(f"Data saved to {csv_file}")


Scraping: https://www.bundesliga.com/en/bundesliga/matchday/2017-2018/1
Selector match-date-header span.day failed: Message: Unable to locate element: match-date-header span.day; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

Selector match-date-header span.day failed: Message: Unable to locate element: match-date-header span.day; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/share

NoSuchWindowException: Message: Browsing context has been discarded
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchWindowError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:679:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.sys.mjs:515:13
assert.open@chrome://remote/content/shared/webdriver/Assert.sys.mjs:147:4
GeckoDriver.prototype.navigateTo@chrome://remote/content/marionette/driver.sys.mjs:859:39
despatch@chrome://remote/content/marionette/server.sys.mjs:318:40
execute@chrome://remote/content/marionette/server.sys.mjs:289:16
onPacket/<@chrome://remote/content/marionette/server.sys.mjs:262:20
onPacket@chrome://remote/content/marionette/server.sys.mjs:263:9
_onJSONObjectReady/<@chrome://remote/content/marionette/transport.sys.mjs:494:20


In [51]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time

# Set up WebDriver
options = webdriver.FirefoxOptions()
options.headless = True
driver = webdriver.Firefox(options=options)

# Define seasons and matchdays
years = range(2017, 2025)
match_days = range(1, 35)
all_matches = []

def get_date_for_match(row):
    """Find the date of a match from its closest match-date-header."""
    try:
        # Check previous elements or container with date
        date_element = row.find_element(By.XPATH, 'preceding::match-date-header[1]')
        return date_element.text.strip()
    except Exception as e:
        print(f"Date extraction failed: {e}")
        return "Unknown Date"

for year in years:
    next_year = year + 1
    for week in match_days:
        url = f'https://www.bundesliga.com/en/bundesliga/matchday/{year}-{next_year}/{week}'
        driver.get(url)
        print(f"Scraping: {url}")

        try:
            # Wait for the main content
            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'fixturescomponent'))
            )
            match_rows = driver.find_elements(By.CSS_SELECTOR, '.matchRow')
            
            for row in match_rows:
                try:
                    date = get_date_for_match(row)
                    home_team = row.find_element(By.CSS_SELECTOR, '#fixtures > fixturescomponent > div > div:nth-child(2) > a > match-fixture > score-bug > div > div.cell.home > div.tlc').text.strip()
                    away_team = row.find_element(By.CSS_SELECTOR, '#fixtures > fixturescomponent > div > div:nth-child(2) > a > match-fixture > score-bug > div > div.cell.away > div.tlc').text.strip()
                    home_score = row.find_element(By.CSS_SELECTOR, 'score-bug .cell.home .score').text.strip()
                    away_score = row.find_element(By.CSS_SELECTOR, 'score-bug .cell.away .score').text.strip()
                    result = f"{home_score}-{away_score}"
                    
                    all_matches.append({
                        'season': f"{year}-{next_year}",
                        'match_day': week,
                        'date': date,
                        'home_team': home_team,
                        'away_team': away_team,
                        'result': result
                    })
                except Exception as e:
                    print(f"Failed to extract match: {e}")
        except Exception as e:
            print(f"Failed to load page or content: {e}")

        time.sleep(1)

driver.quit()

# Save to CSV
csv_file = 'bundesliga_full_data.csv'
csv_columns = ['season', 'match_day', 'date', 'home_team', 'away_team', 'result']

with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=csv_columns)
    writer.writeheader()
    writer.writerows(all_matches)

print(f"Data saved to {csv_file}")


Scraping: https://www.bundesliga.com/en/bundesliga/matchday/2017-2018/1
Failed to extract match: Message: Unable to locate element: #fixtures > fixturescomponent > div > div:nth-child(2) > a > match-fixture > match-team:nth-child(1) > div > div.name.d-none.d-lg-none.d-xl-none.d-md-block; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16

Failed to extract match: Message: Unable to locate element: #fixtures > fixturescomponent > div > div:nth-child(2) > a > match-fixture > match-team:nth-child(1) > div > div.name.d-none.d-lg-none.d-xl-none.d-md-block; For documentation on this error, please visit: h

NoSuchWindowException: Message: Browsing context has been discarded
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchWindowError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:679:5
assert.that/<@chrome://remote/content/shared/webdriver/Assert.sys.mjs:515:13
assert.open@chrome://remote/content/shared/webdriver/Assert.sys.mjs:147:4
GeckoDriver.prototype.navigateTo@chrome://remote/content/marionette/driver.sys.mjs:859:39
despatch@chrome://remote/content/marionette/server.sys.mjs:318:40
execute@chrome://remote/content/marionette/server.sys.mjs:289:16
onPacket/<@chrome://remote/content/marionette/server.sys.mjs:262:20
onPacket@chrome://remote/content/marionette/server.sys.mjs:263:9
_onJSONObjectReady/<@chrome://remote/content/marionette/transport.sys.mjs:494:20
